In [1]:
#import course;course.header()

# Find 10 closest cities to a given latidue and longitude point on earth

Use any data structure that you can get your hands on and load the cities data. Write a function that creates the data structure withthe following function signature

In [2]:
def create_data_stucture(map_data_file):
    """Create a data structure from a data_file that allows rapid quering positions on the map

    Args:
        map_data_file (Path) csv file with the following columns:
            city,lat,lng,country,iso3,local_name,population,continent

    Returns
        data_structure that can be used with find_closest_cities

    HINT:
        BSP buildup using recursive functions
    """

In [3]:
# Daten einlesen, Dictionary, City zu Tupel aus Koordinaten

cities_dict = {}

with open("..\..\data\cities.csv", "r", encoding="utf8") as data:
    next(data)
    for line in data:
        if line[0] == '"':
            splitted_line = line.split(",")
            cities_dict[splitted_line[0]] = (splitted_line[2], splitted_line[3])
        else:
            splitted_line = line.split(",")
            cities_dict[splitted_line[0]] = (splitted_line[1], splitted_line[2]) 


In [4]:
#print(cities_dict)

In [5]:
# Methode um Distanz zwischen Punkt und Stadt zu berechnen (ohne Erdkrümmung)
import numpy as np

def calculate_distance(city, point):
    lat_city = cities_data[city][0]
    lng_city = cities_data[city][1]
    difference_latidue = abs(lat_city - point[0])
    difference_longitude = abs(lng_city - point[1])
    if difference_longitude > 180:
        difference_longitude = 360 - difference_longitude
    distance = np.sqrt(difference_longitude**2 + difference_latidue**2)


In [7]:
# Liste, welche die ersten 10 Städte speichert, eine Neue hinzufügt, und dann die 11. Stadt löscht

point = (0.000, 0.000)
print(geopy.distance.distance(cities_dict["Tokyo"], point))

distances_dict = {}

for key in cities_dict:
    distances_dict[key] = geopy.distance.distance(cities_dict[key], point)


14274.268873103454 km


In [8]:
distance_dict_sorted = {key: value for key, value in sorted(distances_dict.items(), key= lambda item: item[1])}

import itertools
nearest_dict = dict(itertools.islice(distance_dict_sorted.items(), 10))
nearest_list = list(nearest_dict.keys())

print(nearest_list)

['Sekondi', 'Cape Coast', 'Winneba', 'Accra', 'Tema', 'Palé', 'Koforidua', 'Aflao', 'Lomé', 'Aboisso']


Write a query function with the following signature

In [9]:
def find_closest_cities(point, data_structure):
    """Find ten close cities to the given point on the world given the data structure
    
    Args:
        point (tuple of floats) latitude and longitude
    
        data_structure (you choose) to be queries using the point

    Returns:
        list of 10 cities that are closest to the point 
    """
    return